In [1]:
import requests
from bs4 import BeautifulSoup as bs
import time
import re
import random
import pandas as pd

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
    'Cookie':'cors_js=1; _gcl_au=1.1.1386231971.1542590964; _ga=GA1.2.1219686185.1542590966; zz_cook_tms_seg1=1; zz_cook_tms_em=1; _smt_uid=5bf2159c.4434b646; _jc_uid=5bf2159c.4434b646; zz_cook_tms_seg3=8; zz_cook_tms_ed=1; esadm=02UmFuZG9tSVYkc2RlIyh9YbxZGyl9Y5%2BPgn%2Bhd7usVwBeq6nf%2FNucckHYDKGWVmCxcGDrxLJ20EU%3D; BJS=-; _gid=GA1.2.2022348854.1543543770; has_preloaded=1; utag_main=v_id:016740e86def001dd8476b976c090307300b606b0086e$_sn:7$_ss:0$_st:1543550540575$4split:2$4split2:0$ses_id:1543548344472%3Bexp-session$_pn:2%3Bexp-session; header_signin_prompt=1; _gat=1; vpmss=1; lastSeen=0; bkng=11UmFuZG9tSVYkc2RlIyh9YfDNyGw8J7nzPnUG3Pr%2Bfv5iyz1NtopREMUfqnd89duxkGqJP5Vws91G%2BT8YvGIklkwFxUEY38QEhBLutyrqzXaP%2BLRKW88PX1zi5q7N17%2FIoY5odI301Lo7MNV25aUW4w6NF1ZzE4ZzIP5uRL9MhEM0NLpOIOUSvGfjlL16zp2ARDZs72vel54UghJk6vy0Zg%3D%3D'
            }


In [2]:
# 确定要爬取的页面列表，及入住离开日期。
# year = 2018   # 此处调整日期
# month = 11
# month_day_in = 24
# month_day_out = 25
url1 = 'https://www.booking.com/searchresults.zh-cn.html?label=gen173nr-1DCAEoggI46AdIM1gEaDGIAQGYASu4ARfIAQzYAQPoAQGIAgGoAgM;sid=06fa862e2e991308763964858530ab51;checkin_monthday=30&checkin_year_month=2018-11&checkout_monthday=01&checkout_year_month=2018-12&dest_id=-1924465&dest_type=city&from_history=1&group_adults=2&group_children=0&highlighted_hotels=65656&lsuihh=1&no_rooms=1&order=popularity&si=ad&si=ai&si=ci&si=co&si=di&si=la&si=re&;sh_position=1'
url_page_list = []
for i in range(34):  # 此处调整页码
    offset = i * 30
    url_temp = url1 + '&offset='+ str(offset)
    url_page_list.append(url_temp)
print(url_page_list[-1])



https://www.booking.com/searchresults.zh-cn.html?label=gen173nr-1DCAEoggI46AdIM1gEaDGIAQGYASu4ARfIAQzYAQPoAQGIAgGoAgM;sid=06fa862e2e991308763964858530ab51;checkin_monthday=30&checkin_year_month=2018-11&checkout_monthday=01&checkout_year_month=2018-12&dest_id=-1924465&dest_type=city&from_history=1&group_adults=2&group_children=0&highlighted_hotels=65656&lsuihh=1&no_rooms=1&order=popularity&si=ad&si=ai&si=ci&si=co&si=di&si=la&si=re&;sh_position=1&offset=990


In [3]:

content_score = r'<div.*?data-ratings=.*?;(\d.\d|\d);.*?;(\d.\d|\d);.*?;(\d.\d|\d);.*?;(\d.\d|\d);.*?;(\d.\d|\d);.*?;(\d.\d|\d);.*?;(\d.\d|\d);' # 此处用正则
content_name = r'<span class="sr-hotel__name.*?>\s(.*?)\s'
content_review = r'<div class="bui-review-score__text">\s(\d+)\w'
content_price = r'<strong.*?availprice.*?b>\s(\d+)\w'
content_zonghe = r'bui-review-score__content.*?aria-label="(\w+)"'
content_distance = r'class="distfromdest jq_tooltip".*?>\w+(\d+)\skm'

def get_info(url_page_list):
    #七项评分
    info_list_score_1 = []
    info_list_score_2 = []
    info_list_score_3 = []
    info_list_score_4 = []
    info_list_score_5 = []
    info_list_score_6 = []
    info_list_score_7 =[]
    info_list_name = []
    info_list_review = []
    info_list_price = []
    info_list_zonghe = []
    info_list_distance=[]
    info_list = []
    for url in url_page_list:
        time.sleep(random.choice([2,3]))
        response = requests.get(url,headers=headers)
        soup = bs(response.text,'lxml')
        hotel_list = soup.select('#hotellist_inner div.sr_item_content.sr_item_content_slider_wrapper ') # 这个. 很重要
        for i in range(len(hotel_list)):
            score = re.compile(content_score,re.S).findall(str(hotel_list[i]))
            name = re.compile(content_name,re.S).findall(str(hotel_list[i]))
            review = re.compile(content_review,re.S).findall(str(hotel_list[i]))
            price = re.compile(content_price,re.S).findall(str(hotel_list[i]))
            zonghe = re.compile(content_zonghe,re.S).findall(str(hotel_list[i]))
            distance = re.compile(content_distance,re.S).findall(str(hotel_list[i]))
            #创建列表 填充缺失值
            #七项评分
            if score:
                info_list_score_1.append(score[0][0])
                info_list_score_2.append(score[0][1])
                info_list_score_3.append(score[0][2])
                info_list_score_4.append(score[0][3])
                info_list_score_5.append(score[0][4])
                info_list_score_6.append(score[0][5])
                info_list_score_7.append(score[0][6])
            else:
                info_list_score_1.append('NaN')
                info_list_score_2.append('NaN')
                info_list_score_3.append('NaN')
                info_list_score_4.append('NaN')
                info_list_score_5.append('NaN')
                info_list_score_6.append('NaN')
                info_list_score_7.append('NaN')
            #酒店名 评论数 价格
            info_list_name.append(name[0])

            if review:
                info_list_review.append(review[0])
            else:
                info_list_review.append('NaN')

            if price:
                info_list_price.append(price[0])
            else:
                info_list_price.append('NaN')

            if zonghe:
                info_list_zonghe.append(zonghe[0])
            else:
                info_list_zonghe.append('NaN')
                
            if distance:
                info_list_distance.append(distance[0])
            else:
                info_list_distance.append('NaN')

            info_list.append(info_list_name)
            info_list.append(info_list_score_1)
            info_list.append(info_list_score_2)
            info_list.append(info_list_score_3)
            info_list.append(info_list_score_4)
            info_list.append(info_list_score_5)
            info_list.append(info_list_score_6)
            info_list.append(info_list_score_7)
            info_list.append(info_list_review)
            info_list.append(info_list_price)
            info_list.append(info_list_zonghe)
            info_list.append(info_list_distance)
    return info_list

In [4]:

def DownLoadInfo(a):
    test1 = pd.DataFrame(columns = ['酒店名'],data=a[0])
    test1['清洁程度'] = a[1]
    test1['舒适程度'] = a[2]
    test1['位置'] = a[3]
    test1['设施/服务'] = a[4]
    test1['员工素质'] = a[5]
    test1['性价比'] = a[6]
    test1['免费WiFi'] = a[7]
    test1['评论数'] = a[8]
    test1['价格'] = a[9]
    test1['综合评价'] = a[10]
    test1['离中心地区距离'] = a[11]
    return test1

In [5]:
a = get_info(url_page_list)
test1 = DownLoadInfo(a)
test1.to_csv('test1.csv')